```{contents}
```
## **Conversation State in LangGraph**

In LangGraph, **Conversation State** is the structured, persistent memory object that represents **everything the system knows about an ongoing interaction**.
It enables **multi-turn reasoning, agent coordination, long-running workflows, and recovery across executions**.

At a systems level, conversation state is the **single source of truth** driving execution, routing, memory, and behavior.

---

### **1. Formal Definition**

> **Conversation State** = Typed, versioned, persistent data structure that is **read and updated by every node** during graph execution.

It acts as the **memory + control plane** of the entire workflow.

---

### **2. Why Conversation State Is Necessary**

Without conversation state, workflows are:

* Stateless
* Single-turn
* Fragile
* Non-recoverable

With conversation state, LangGraph supports:

| Capability          | Enabled by State |
| ------------------- | ---------------- |
| Multi-turn dialogue | Yes              |
| Tool coordination   | Yes              |
| Agent collaboration | Yes              |
| Looping & planning  | Yes              |
| Checkpoint & resume | Yes              |
| Human-in-the-loop   | Yes              |
| Long-running tasks  | Yes              |

---

### **3. Structure of Conversation State**

Conversation state is **explicitly typed**.

```python
from typing import TypedDict, List

class State(TypedDict):
    messages: List[dict]
    user_goal: str
    plan: str
    last_action: str
    tool_results: dict
    done: bool
```

Each node:

* **Reads** the current state
* **Returns partial updates**
* LangGraph **merges updates deterministically**

---

### **4. How State Flows Through the Graph**

```
User Input → State → Node A → Updated State → Node B → Updated State → ...
```

No node communicates directly with another.
All coordination happens through **state transitions**.

---

### **5. Message History as Conversation State**

A common pattern:

```python
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: list

def chat_node(state):
    response = llm.invoke(state["messages"])
    return {"messages": add_messages(state["messages"], response)}
```

This allows the model to reason over the **entire conversation context**.

---

### **6. State Persistence & Recovery**

LangGraph supports durable state using **checkpoint stores**.

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver("conversations.db")
graph = builder.compile(checkpointer=checkpointer)
```

Capabilities enabled:

| Feature          | Description    |
| ---------------- | -------------- |
| Crash recovery   | Resume exactly |
| Replay           | Debugging      |
| Auditing         | Full history   |
| Long-term memory | Yes            |

---

### **7. Conversation State in Cyclic Graphs**

State controls looping behavior:

```python
def router(state):
    if state["done"]:
        return END
    return "reason"
```

State is the **control signal** for iteration.

---

### **8. Multi-Agent Conversation State**

Agents collaborate **through shared state**.

| Agent      | State Fields |
| ---------- | ------------ |
| Planner    | plan         |
| Researcher | evidence     |
| Executor   | result       |
| Critic     | feedback     |

This allows **structured agent communication**.

---

### **9. Production Design Principles**

| Principle       | Rationale            |
| --------------- | -------------------- |
| Typed schema    | Safety & correctness |
| Partial updates | Efficiency           |
| Immutability    | Debugging & replay   |
| Persistence     | Fault tolerance      |
| Versioning      | Upgrades             |
| Encryption      | Security             |

---

### **10. Mental Model**

Conversation state in LangGraph behaves like:

> **A distributed control system's memory register**

It is both:

* **Memory** (what the system knows)
* **Control logic** (how the system decides)

---

### **11. Comparison**

| Traditional Chat | LangGraph              |
| ---------------- | ---------------------- |
| Prompt string    | Structured state       |
| Hidden memory    | Explicit memory        |
| No recovery      | Durable recovery       |
| Single LLM       | Multi-agent system     |
| Linear flow      | Cyclic autonomous flow |


### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: List[dict]
    plan: str
    done: bool

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def reason(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": add_messages(state["messages"], response),
        "plan": "refine",
        "done": False
    }

def reflect(state: State):
    critique = llm.invoke([
        {"role": "system", "content": "Check if the answer is complete."},
        *state["messages"]
    ])
    messages = add_messages(state["messages"], critique)
    done = "complete" in critique.content.lower()
    return {"messages": messages, "done": done}

from langgraph.graph import END

def router(state: State):
    return END if state["done"] else "reason"

In [2]:
from langgraph.graph import StateGraph

builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("reflect", reflect)

builder.set_entry_point("reason")
builder.add_edge("reason", "reflect")

builder.add_conditional_edges("reflect", router, {
    "reason": "reason",
    END: END
})

from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [5]:
input_state = {
    "messages": [{"role": "user", "content": "Explain neural networks simply."}],
    "plan": "",
    "done": False
}

config = {"configurable": {"thread_id": "1"}, "recursion_limit": 50}
result = graph.invoke(input_state, config)


In [6]:
result

{'messages': [HumanMessage(content='Explain neural networks simply.', additional_kwargs={}, response_metadata={}, id='c93358c2-4e99-42a1-8f56-05f5a2d468c7'),
  AIMessage(content='Sure! Neural networks are a type of computer program modeled after the way our brains work. They are designed to recognize patterns and learn from data.\n\nHere\'s a simple breakdown of how they work:\n\n1. **Neurons**: Think of a neural network as a web of interconnected units called neurons. Just like human neurons, these units take in information, process it, and pass on the result.\n\n2. **Layers**: Neural networks consist of layers. The first layer (input layer) receives data, like images or text. The middle layers (hidden layers) process this information, and the last layer (output layer) gives the final results, such as classifications or predictions.\n\n3. **Weights and Biases**: Each connection between neurons has a weight, which determines the importance of the signal being passed. There are also bia